<a href="https://colab.research.google.com/github/bxck75/colab_projects/blob/master/audio_video_swapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Swapper

In [0]:
#@title Setup the swap
#@markdown video to get the dancer video from
DanceVideo='https://www.youtube.com/watch?v=vdJcJAO8lrE' #@param {type:"string"}
#@markdown video to get the audio from
AudioVideo='https://www.youtube.com/watch?v=PKkqD4HPimI' #@param {type:"string"}
DanceSpeed='0.5' #@param [1.0, 0.25, 0.50, 0.75, 0.1] {allow-input: true}
AudioSpeed='1.20' #@param [1.0, 1.25, 1.50, 1.75, 2.0] {allow-input: true}
Trim_Start_Dance= 36 #@param {type: "slider", min: 0, max: 200}
Trim_Start_Audio= 0 #@param {type: "slider", min: 0, max: 200}
Final_output_name = 'swapped_video' #@param {type: "string"}

#widgets

All input widgets!
sys.exit()
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = 'name'  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 179  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---*italicized text* 


#Run

In [0]:
#setup imports
!pip install youtube-dl
!pip install ffmpeg moviepy imageio
import moviepy,os
import moviepy.editor as mp
import imageio 
from moviepy.editor import VideoFileClip, AudioFileClip,concatenate_audioclips,concatenate_videoclips,clips_array, vfx
from moviepy.video.fx import crop
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def Trim_video(t,tr,start=0,stop=100):  
  # cut clip if neeed
  ffmpeg_extract_subclip(t, start, stop, targetname=tr) 
# make folder for extraction
if not os.path.exists('/content/audioswapper'):
  os.mkdir('/content/audioswapper',777)
if not os.path.exists('/content/final_result'):
  os.mkdir('/content/final_result',777)
#rip the videos  
!youtube-dl --format mp4 {DanceVideo} --output '/content/audioswapper/konijn.mp4'
!youtube-dl --format mp4 {AudioVideo} --output '/content/audioswapper/sjeik.mp4'
# trim the audio  
Trim_video('/content/audioswapper/sjeik.mp4','/content/audioswapper/sjeik_trimmed.mp4',Trim_Start_Audio,630)
# trim the video
Trim_video('/content/audioswapper/konijn.mp4','/content/audioswapper/konijn_trimmed.mp4',Trim_Start_Dance,630)

# speed up dance vid
print('workig on speeding')
!ffmpeg -y -i "/content/audioswapper/konijn_trimmed.mp4" \
  -filter_complex "[0:v]setpts={DanceSpeed}*PTS[v];[0:a]atempo=1.0[a]" -map "[v]" \
  -map "[a]" "/content/audioswapper/konijn_up_speed.mp4"
!ffmpeg -y -i "/content/audioswapper/sjeik_trimmed.mp4" \
  -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo={AudioSpeed}[a]" -map "[v]" \
  -map "[a]" "/content/audioswapper/sjeik_up_speed.mp4"

# split the audio from de trimmed video
videoclip = VideoFileClip('/content/audioswapper/sjeik_up_speed.mp4')
audioclip = videoclip.audio
audioclip.write_audiofile('/content/audioswapper/sjeik.mp3')

# combine the two
video = VideoFileClip("/content/audioswapper/konijn_up_speed.mp4")
video_with_new_audio = video.set_audio(AudioFileClip("/content/audioswapper/sjeik.mp3")) 
# video_with_new_audio.write_videofile("/content/final_result/sjeik_konijn.mp4", fps=60, codec="mpeg4")
video_with_new_audio.write_videofile("/content/final_result/"+Final_output_name+".mp4")
!rm -r '/content/audioswapper'